In [1]:
import os
import torch

In [2]:
ds = "Military objects in military environments/Dataset"
# ds = f"military drones images"

models = f"/mnt/d/23b6034/FYP/models/DETR-4"
train_dataset = f"/mnt/d/23b6034/FYP/datasets/{ds}/train"
test_dataset = f"/mnt/d/23b6034/FYP/datasets/{ds}/test"
val_dataset = f"/mnt/d/23b6034/FYP/datasets/{ds}/valid"

In [3]:
# Set TORCH_HOME to your desired location
os.environ['TORCH_HOME'] = r'D:\23b6034\FYP\models'
model2 = torch.hub.load('facebookresearch/detr:main', 'detr_resnet50', pretrained=True)
model2.class_embed = torch.nn.Linear(model2.class_embed.in_features, 4)



Using cache found in D:\23b6034\FYP\models/hub/facebookresearch_detr_main
/mnt/d/23b6034/FYP/vengeance/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/mnt/d/23b6034/FYP/vengeance/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
from tqdm import tqdm
import torch
import os
import torch.optim as optim # Import the optim module
device = "cuda"
num_epochs = 300
# Move the model to the device (GPU if available)
model2.to(device)

# Define a directory to save checkpoints
checkpoint_dir = models
os.makedirs(checkpoint_dir, exist_ok=True) # just make the dir if not there

start_epoch = 0
optimizer = None  # Initialize optimizer to None

# Check for existing checkpoints to resume training
checkpoints = [f for f in os.listdir(checkpoint_dir) if f.endswith('.pth')]
if checkpoints:
    print("Found existing checkpoints. Resuming training.")
    # Find the latest checkpoint
    latest_checkpoint = max([os.path.join(checkpoint_dir, f) for f in checkpoints], key=os.path.getctime) # ---- if folder has .pth file, use the one trained latest
    print(f"Loading checkpoint from: {latest_checkpoint}")
    model2.load_state_dict(torch.load(latest_checkpoint, map_location=torch.device('cpu'))) # ------------------ load the thing
    start_epoch = int(latest_checkpoint.split('_')[-1].split('.')[0]) # ---------------------------------------- set the epoch to the last trained epoch
    print(f"Resuming from epoch {start_epoch + 1}")

    optimizer = optim.AdamW(model2.parameters(), lr=1e-5)

else:
    print("No existing checkpoints found. Starting training from scratch.")
    optimizer = optim.AdamW(model2.parameters(), lr=1e-5)


Found existing checkpoints. Resuming training.
Loading checkpoint from: /mnt/d/23b6034/FYP/models/DETR-4/model_epoch_163.pth
Resuming from epoch 164


In [10]:
import torchvision.transforms as transforms
import torch
import os
from torch.utils.data import DataLoader


ds = "Military objects in military environments/Dataset"
# ds = f"military drones images"

models = f"/mnt/d/23b6034/FYP/models/DETR-4"
train_dataset = f"/mnt/d/23b6034/FYP/datasets/{ds}/train"
test_dataset = f"/mnt/d/23b6034/FYP/datasets/{ds}/test"
val_dataset = f"/mnt/d/23b6034/FYP/datasets/{ds}/valid"

class DroneDataset(torch.utils.data.Dataset):
    def __init__(self, image_dir, label_dir, classes, transform=None):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.classes = classes
        self.transform = transform
        self.image_filenames = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        img_name = self.image_filenames[idx]
        img_path = os.path.join(self.image_dir, img_name)
        label_path = os.path.join(self.label_dir, os.path.splitext(img_name)[0] + '.txt') # Assuming label files have the same name as images with a .txt extension

        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        boxes = []
        labels = []
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f.readlines():
                    class_id, center_x, center_y, width, height = map(float, line.strip().split())
                    # Convert YOLO format (center_x, center_y, width, height) to [x_min, y_min, x_max, y_max]
                    # Assuming coordinates are normalized (0 to 1)
                    x_min = center_x - width / 2
                    y_min = center_y - height / 2
                    x_max = center_x + width / 2
                    y_max = center_y + height / 2
                    boxes.append([x_min, y_min, x_max, y_max])
                    labels.append(int(class_id)) # Assuming class_id is an integer

        target = {}
        target["boxes"] = torch.as_tensor(boxes, dtype=torch.float32)
        target["labels"] = torch.as_tensor(labels, dtype=torch.int64)

        return image, target
    
classes = {
    0 : "Tank",
    1 : "drone",
    2 : "people",
    3 : "soldier"
}
#tank drone people soldier

# Re-define the classes dictionary


# Define image and label directory paths
train_image_dir = train_dataset + "/images"
train_label_dir = train_dataset + "/labels"
val_image_dir = val_dataset + "/images"
val_label_dir = val_dataset + "/labels"
test_image_dir = test_dataset + "/images"
test_label_dir = test_dataset + "/labels"

# Define a suitable transform
# For DETR, a common transform resizes the image and normalizes it
transform = transforms.Compose([
    transforms.Resize((320, 320)),
    transforms.ToTensor(),
    # Add more transforms if needed, e.g., resizing to a fixed size expected by DETR
    # transforms.Resize((800, 800)),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Instantiate the DroneDataset for each set
train_drone_dataset = DroneDataset(train_image_dir, train_label_dir, classes, transform=transform)
val_drone_dataset = DroneDataset(val_image_dir, val_label_dir, classes, transform=transform)
test_drone_dataset = DroneDataset(test_image_dir, test_label_dir, classes, transform=transform)


# Instantiate DataLoader objects for each dataset
batch_size = 1  # You can adjust this based on your GPU memory
# Added num_workers for parallel data loading
train_dataloader = DataLoader(train_drone_dataset, batch_size=batch_size, shuffle=True, collate_fn=lambda x: tuple(zip(*x)), num_workers=0)
val_dataloader = DataLoader(val_drone_dataset, batch_size=batch_size, shuffle=False, collate_fn=lambda x: tuple(zip(*x)), num_workers=0)
test_dataloader = DataLoader(test_drone_dataset, batch_size=batch_size, shuffle=False, collate_fn=lambda x: tuple(zip(*x)), num_workers=0)

# print("DataLoaders for train, validation, and test datasets instantiated with num_workers.")

In [12]:
# Performance Metrics for Object Detection (DETR)
# This cell computes mAP, precision, recall, F1 score, and per-class AP using torchmetrics

import torch
import numpy as np
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from collections import defaultdict
import danial_loader

# Initialize the metric
map_metric = MeanAveragePrecision(iou_type="bbox", class_metrics=True)

def evaluate_model_metrics(model, dataloader, device, classes, confidence_threshold=0.5):
    model.eval()
    all_outputs = []
    all_targets = []
    with torch.no_grad():
        for images, targets in dataloader:
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            outputs = model(images)
            # Convert outputs to torchmetrics format
            batch_preds = []
            batch_targets = []
            for i in range(len(images)):
                # Predictions
                pred_logits = outputs['pred_logits'][i]
                pred_boxes = outputs['pred_boxes'][i]
                scores, labels = torch.max(torch.softmax(pred_logits, dim=-1), dim=-1)
                keep = scores > confidence_threshold
                batch_preds.append({
                    'boxes': pred_boxes[keep].detach().cpu(),
                    'scores': scores[keep].detach().cpu(),
                    'labels': labels[keep].detach().cpu()
                })
                # Ground truth
                batch_targets.append({
                    'boxes': targets[i]['boxes'].detach().cpu(),
                    'labels': targets[i]['labels'].detach().cpu()
                })
            all_outputs.extend(batch_preds)
            all_targets.extend(batch_targets)
    # Compute metrics
    map_metric.reset()
    map_metric.update(all_outputs, all_targets)
    results = map_metric.compute()
    print("\n--- Detection Metrics ---")
    print(f"mAP@0.5: {results['map_50']:.4f}")
    print(f"mAP@0.5:0.95: {results['map']:.4f}")
    # print(f"Precision: {results['precision']:.4f}")
    # print(f"Recall: {results['recall']:.4f}")
    # print(f"F1 Score: {results['f1']:.4f}")
    print("Per-class AP:")
    for i, ap in enumerate(results['map_per_class']):
        class_name = classes.get(i, str(i))
        print(f"  {class_name}: {ap:.4f}")
    return results

# Example usage:
# dataloader = danial_loader.load_train()
metrics = evaluate_model_metrics(model2, val_dataloader, device, 4, confidence_threshold=0.5)
# You can also run on val_dataloader for validation metrics

NameError: name 'Image' is not defined